In [1]:
import os
import getpass
import sqlite3
import pandas as pd
import time
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain_chroma import Chroma
import chromadb
import yaml

In [2]:
conn = sqlite3.connect("data/database.db")
tickers = pd.read_sql_query("SELECT distinct Security FROM master_ticker", conn)["Security"].tolist()

In [3]:
with open("keys.yaml") as keys:
    try:
        api_keys = yaml.safe_load(keys)
    except yaml.YAMLError as exc:
        print(exc)

In [7]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = api_keys["hf_model"]

llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    temperature=0,
    repetition_penalty=1.03,
)

chat_model = ChatHuggingFace(llm=llm)


c:\Users\feder\anaconda3\envs\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\feder\.cache\huggingface\token
Login successful


### PDF Loader

In [8]:
loader = PyPDFDirectoryLoader("data/wikipedia/")
docs = loader.load()

In [ ]:
from langchain_community.vectorstores import FAISS
faiss_index = FAISS.from_documents(docs, HuggingFaceEmbeddings())
pages = faiss_index.similarity_search("Tell me everything about ACN")

In [13]:
from langchain_core.messages import (
    HumanMessage,
    SystemMessage)

responses = []
messages = [
    SystemMessage(content="""You know about the SPY500 companies. Only Answer the headquarter of the company mentioned. Answer shortly and concise. 
                  When prompted answer in this format: City, Country"""),
    HumanMessage(
        content=str(tickers)
    ),]
ai_msg = chat_model.invoke(messages)
ai_msg

AIMessage(content='City, Country: Alphabet Inc. (Class A) - Mountain View, USA\nCity, Country: Alphabet Inc. (Class C) - Mountain View, USA\nCity, Country: Amazon - Seattle, USA\nCity, Country: Ameriprise Financial - Minneapolis, USA\nCity, Country: Amgen - Thousand Oaks, USA\nCity, Country: AT&T - Dallas, USA\nCity, Country: AutoZone - Memphis, USA', response_metadata={'token_usage': ChatCompletionOutputUsage(completion_tokens=100, prompt_tokens=3065, total_tokens=3165), 'model': '', 'finish_reason': 'length'}, id='run-83ac5a1b-9afa-4116-87bb-4fe12fc7a2fe-0')

In [ ]:
truth = pd.read_sql_query("SELECT * FROM master_ticker", conn)
truth["ZL_headquarters"] = responses

'Saint Paul, United States'

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings())

retriever = vectorstore.as_retriever()

NameError: name 'docs' is not defined